In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pytorch_forecasting==0.9.0 #Instalando pytorch

In [ ]:
#Importando bibliotecas
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch
import torch.nn as nn



import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile



import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_item = pd.read_csv("/kaggle/input/datahfinal/meta-item.csv")
df_sales = pd.read_csv('/kaggle/input/datahfinal/sales.csv')
df_store = pd.read_csv('/kaggle/input/datahfinal/meta-store.csv')
df_sub = pd.read_csv('/kaggle/input/datahfinal/submission_sample.csv')
df_item.describe()

In [ ]:
df_sales['DATE'] = pd.to_datetime(df_sales['DATE'])
cols = ['DATE', 'S100', 'S101', 'S102', 'S103', 'I100', 'I101', 'I102', 'I103', 'C100', 'C101', 'QTT']

df_merge = df_sales
df_mesclado = df_merge.merge(df_store, how='left',on='S100')
df_mesclado2 = df_mesclado.merge(df_item, how='left', on='I100')
cols = ['DATE', 'S100', 'S101', 'S102', 'S103', 'I100', 'I101', 'I102', 'I103', 'C100', 'C101', 'QTT']
df_compl = df_mesclado2[cols]
df_compl.head()

In [ ]:
#Diminuindo espaço do dataframe
cols = ['S100', 'S101', 'S102', 'S103', 'I100', 'I101', 'I102', 'I103', 'C100', 'C101', 'QTT']

for i in cols:
    df_compl[i] = pd.to_numeric(df_compl[i],downcast="integer")

df_compl.info()

In [ ]:
df_compl.describe()

In [ ]:
df_compl.tail() 

In [ ]:
df_sub.head()

In [ ]:
df_compl['Mes']= df_compl.DATE.dt.month.astype(str).astype('category')

In [ ]:
#Train and Test dataset
df_train = df_compl.loc[(df_compl['DATE']>='2020-07-01')&(df_compl['DATE']<'2021-12-27')]
df_test = df_compl.loc[df_compl['DATE']>='2021-10-27']
print("Dataset treino shape: ", df_train.shape)
print("Dataset teste shape: ",df_test.shape)

**Função criar para Time_index para o tft**

In [ ]:
def criatimeidx(df):
    for loja in range(0,31):
        for item in range(0,96):
            df_temp = df.loc[((df['S100']==loja)&(df['I100']==item))]
            i=0
            for index in df_temp.index:
                i = i+1
                df_temp['Time_idx'][index]=i
            df.loc[((df['S100']==loja)&(df['I100']==item)),'Time_idx']=df_temp['Time_idx']
    return df

In [ ]:
#criando e preenchendo coluna time-index do conjunto treino
df_train['Time_idx']=0
df_train = criatimeidx(df_train)
df_train.head()

In [ ]:
df_test['Time_idx']=0
df_test = criatimeidx(df_test)
df_test.head()

In [ ]:
df_train.info()

Referência: https://pytorch-forecasting.readthedocs.io/en/latest/tutorials/stallion.html

In [ ]:
#Criando TimeSeriesDataset e setando parametros
max_prediction_length = 1
max_encoder_length = 10
training_cutoff = df_train['Time_idx'].max()-max_prediction_length

training = TimeSeriesDataSet(
    df_train[lambda x: x.Time_idx <= training_cutoff],
    time_idx='Time_idx',
    target="QTT",
    group_ids=["S100", "I100"],
    min_encoder_length=1,  
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=["S101","S102","S103","I101","I102","I103"],
    time_varying_known_categoricals=["Mes"],  
    time_varying_known_reals=["Time_idx","C100","C101"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=['QTT'],
    target_normalizer=GroupNormalizer(groups=["S100","I100"],transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
    
)

In [ ]:
training.get_parameters()

In [ ]:
#Criando dados de treino e validação
validation = TimeSeriesDataSet.from_dataset(training, df_train, predict=True, stop_randomization=True)

batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
#(actuals - baseline_predictions).abs().mean().item()
criterion = nn.MSELoss()
torch.sqrt(criterion(actuals,baseline_predictions))

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    #gpus=1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=128,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=2,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=64,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=0.1,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.0001, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs") 

trainer = pl.Trainer(
    max_epochs=1,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.02,
    limit_train_batches=30,  
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.02,
    hidden_size=128,
    attention_head_size=2,
    dropout=0.1756,
    hidden_continuous_size=64,
    output_size=1, 
    loss=pytorch_forecasting.metrics.RMSE(),
    log_interval=10,  
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
#otimizar os hiperparametros...
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=100,
    max_epochs=1,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte root mean squared error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
val_predictions = best_tft.predict(val_dataloader)

In [ ]:
type(val_predictions)

**Avaliar Desempenho**

In [ ]:
criterion = nn.MSELoss()
torch.sqrt(criterion(actuals,val_predictions))

In [ ]:
plot_predictions,x=best_tft.predict(val_dataloader,mode= "raw", return_x=True)

In [ ]:
for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(x, plot_predictions, idx=idx, add_loss_to_title=True)

In [ ]:
interpretation = best_tft.interpret_output(plot_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
#Funcão cria ID para tratar o dataframe de teste e de submissao
def criaID(df):
    df['ID'] = df.S100.astype(str).str.cat(df['I100'].astype(str), sep = '_')
    return df

In [ ]:
def pred_test(df,best_tft):
    idx_maximo = df['Time_idx'].max()
    for i in range(2,idx_maximo+1):
        df_temp = df.loc[df['Time_idx']==i,['ID']].reset_index(drop=True)
        df_temp2 = df.loc[(df['ID'].isin(df_temp['ID']))&(df['Time_idx']<=i)]
        pred = best_tft.predict(df_temp2)
        pred = pred.reshape(-1,)
        previsoes = pd.DataFrame()
        previsoes['QTT2']= np.array(pred).astype('float')
        df.loc[df['Time_idx']==i,'QTT2']=previsoes['QTT2'].values
        #print("Time_idx: ",i)
    return df
    

In [ ]:
def pred_sub(df,best_tft):
    idx_maximo = df['Time_idx'].max()
    for i in range(7,idx_maximo+1):
        df_temp = df.loc[df['Time_idx']==i,['ID']].reset_index(drop=True)
        df_temp2 = df.loc[(df['ID'].isin(df_temp['ID']))&(df['Time_idx']<=i)]
        pred = best_tft.predict(df_temp2)
        pred = pred.reshape(-1,)
        previsoes = pd.DataFrame()
        previsoes['QTT2'] = np.array(pred).astype('float')
        
        df.loc[df['Time_idx']==i,'QTT']=previsoes['QTT2'].values
        if len(df.loc[df['QTT'].isnull(),"ID"])>0:
            df.fillna(1,inplace=True)
            print("Fill NA")
        print("Time_idx: ",i)
    return df

In [ ]:
df_test = criaID(df_test)
df_test.head()

In [ ]:
df_test['QTT2']=1
df_test = pred_test(df_test, best_tft)
df_test.head(10)

In [ ]:
df_test.loc[df_test['QTT2']>15,['ID','DATE','Time_idx','QTT','QTT2']]

In [ ]:
#RMSE treinamento
from sklearn.metrics import mean_squared_error
print("Resultado")
qtt1 = df_test.loc[df_test['Time_idx']>1,'QTT']
qtt2 = df_test.loc[df_test['Time_idx']>1,'QTT2']
np.sqrt(mean_squared_error(qtt1,qtt2))

**Submission**

In [ ]:
df_sub = df_sub['ID'].str.split('_', expand= True)
cols= {0:'DATE',1:'S100',2:'I100',3:'C100',4:'C101'}
df_sub.rename(columns=cols,inplace=True)
df_sub['QTT'] = 1
df_sub.head()

In [ ]:
df_sub.info()

In [ ]:
df_sub[["S100","I100","C100","C101"]] = df_sub[["S100","I100","C100","C101"]].apply(pd.to_numeric)
df_sub.info()

In [ ]:
#Criando dataset de submissao e mesclando datasets
df_sub['DATE'] = pd.to_datetime(df_sub['DATE'])

df_merge = df_sub
df_mesclado = df_merge.merge(df_store, how='left',on='S100')
df_mesclado2 = df_mesclado.merge(df_item, how='left', on='I100')
cols = ['DATE', 'S100', 'S101', 'S102', 'S103', 'I100', 'I101', 'I102', 'I103', 'C100', 'C101', 'QTT']
df_sub= df_mesclado2[cols]
df_sub['Mes']= df_sub.DATE.dt.month.astype(str).astype('category')
df_sub.head()

In [ ]:
# Cria time_idx para dataset de submissao iniciando do 5
def criatimeidx_sub(df):
    for loja in range(0,31):
        for item in range(0,96):
            df_temp = df.loc[((df['S100']==loja)&(df['I100']==item))]
            i=5
            for index in df_temp.index:
                i = i+1
                df_temp['Time_idx'][index]=i
            df.loc[((df['S100']==loja)&(df['I100']==item)),'Time_idx']=df_temp['Time_idx']
    return df

In [ ]:
df_sub['Time_idx']=0
df_sub = criatimeidx_sub(df_sub)
df_sub = criaID(df_sub)
df_sub.head()

**Foi criado um buffer das ultimas 5 vendas para cadas serie temporal e add no arquivo de submissão para iniciar as previsões**

In [ ]:
#Criando ID na dataset de treinamento para extrair o buffer
df_train = criaID(df_train)
df_train.head()

In [ ]:
ids = df_sub['ID'].unique()
df_tempID = pd.DataFrame()
df_tempID['ID'] = ids
for i in (df_tempID['ID']):
    df_tempID.loc[df_tempID['ID']==i,'Time_max'] = df_train.Time_idx.loc[df_train['ID']==i].max()
df_tempID.head()

In [ ]:
df_tempID.loc[df_tempID['Time_max']<5]

In [ ]:
df_tempID.loc[df_tempID['Time_max'].isnull()]

In [ ]:
df_buffer = df_train.loc[df_train['ID']=='0_1'].iloc[-5:]
for i in (df_tempID['ID'][1:]):
    df_buffer = df_buffer.append(df_train.loc[df_train['ID']==i].iloc[-5:])
df_buffer.head()

In [ ]:
df_buffer.sort_values('DATE',inplace=True)
df_buffer.head(20)

In [ ]:
df_buffer = criatimeidx(df_buffer)
df_buffer.head(20)

In [ ]:
df_buffer['Time_idx'].max()

In [ ]:
df_sub2 = df_buffer.append(df_sub)
df_sub2.head()

In [ ]:
df_sub2.fillna(1,inplace=True)

In [ ]:
df_sub2 = pred_sub(df_sub2, best_tft)

In [ ]:
df_sub = df_sub2.loc[df_sub2['DATE']>='2021-12-19']
df_sub.describe()

In [ ]:
df_sub['ID']= df_sub.DATE.astype(str).str.cat(df_sub[["S100","I100","C100","C101"]].astype(str), sep='_')
df_final = df_sub.drop(columns={'DATE','S100','S101','S102','S103','I100','I101','I102','I103','C100','C101','Time_idx'},axis=1)
df_final = df_final[['ID','QTT']]
df_final.head()

In [ ]:
df_final

In [ ]:
df_final.to_csv ('tft_test.csv', index = False, header=True)